# Amazon Fraud Detector - Data Profiler Notebook 


### Dataset Guidance
-------

AWS Fraud Detector models support a flexible schema, enabling you to train models to your specific data and business need. This notebook was developed to help you profile your data and identify potenital issues before you train an AFD model. The following summarizes the minimimum CSV File requirements:

* The files are in CSV UTF-8 (comma delimited) format (*.csv).
* The file should contain at least 10k rows and the following __two__ required fields:  

    * Event timestamp 
    * Fraud label 
    
* The maximum file size is 10 gigabytes (GB).  

* The following dates and datetime formats are supported:
    * Dates: YYYY-MM-DD (eg. 2019-03-21)
    * Datetime: YYYY-MM-DD HH:mm:ss (eg. 2019-03-21 12:01:32) 
    * ISO 8601 Datetime: YYYY-MM-DDTHH:mm:ss+/-HH:mm (eg. 2019-03-21T20:58:41+07:00)

* The decimal precision is up to four decimal places.
* Numeric data should not contain commas and currency symbols. 
* Columns with values that could contain commas, such as address or custom text should be enclosed in double quotes. 



### Getting Started with Data 
-------
The following general guidance is provided to get the most out of your AWS Fraud Detector Model. 

* Gathering Data -  The AFD model requires a minimum of 10k records. We recommend that a minimum of 6 weeks of historic data is collected, though 3 - 6 months of data is preferable.   As part of the process the AFD model partitions your data based on the Event Timestamp such that performance metrics are calculated on the out of sample (latest) data, thus the format of the event timestamp is important. 

  
* Data & Label Maturity: As part of the data gathering process we want to insure that records have had sufficient time to “mature”, i.e. that enough time has passed to insure “non-fraud" and  “fraud” records have been correctly identified. It often takes 30 - 45 days (or more) to correctly identify fraudulent events, because of this it is important to insure that the latest records are at least 30 days old or older.  

  
* Sampling: The AFD training process will sample and partition historic based on event timestamp. There is no need to manually sample the data and doing so may negatively influence your model’s results.  

  
* Fraud Labels:  The AFD model requires that a minimum of 500 observations are identified and labeled as “fraud”. As noted above, fraud label maturity is important. Insure that extracted data has sufficiently matured to insure that fraudulent events have been reliably found. 
  
  
* Custom Fields: the AFD model requires 2 fields: event timestamp and fraud label. The more custom fields you provide the better the AFD model can differentiate between fraud and not fraud.  
  
  
* Nulls and Missing Values: AFD model handles null and missing values, however the percentage of nulls in key fields should be limited. Especially timestamp and fraud label columns should not contain any missing values.   

  
If you would like to know more, please check out the [Fraud Detector's Documentation](https://docs.aws.amazon.com/frauddetector/). 


In [ ]:
from IPython.core.display import display, HTML
from IPython.display import clear_output
display(HTML("<style>.container { width:90% }</style>"))
from IPython.display import IFrame
# ------------------------------------------------------------------
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.display.float_format = '{:.4f}'.format

# -- AWS stuff -- 
import boto3


### Check required packages

In [ ]:
import urllib3
import numpy as np
import pandas as pd
import awswrangler as wr
import boto3
import os
import json
import re
import sys
from jinja2 import Environment, PackageLoader, FileSystemLoader
import itertools
from pandas import DataFrame, Series
import scipy.stats as ss
import logging

#### If you see errors of missing modules, use pip to install those modules

In [ ]:
# !pip install awswrangler
# !pip install Jinja2

### Amazon Fraud Detector Profiling 
-----

from github download and copy the afd_profile.py python program and template directory to your notebook  

<div class="alert alert-info"> <strong> afd_profile.py </strong>

- afd_profile.py - is the python package which will generate your profile report. 

</div>


In [ ]:
# -- get this package from github -- 
import afd_profile

### File & Field Mapping
-----
Simply map your file and field names to the required config values. 

<div class="alert alert-info"> <strong> Map the Required fields </strong>

- CSVFilePath: S3 path to your CSV file. E.g. s3://mybucket/myfile.csv
- EventTimestampColumn: Column name of event timestamp in your CSV file. This is a mandatory column for AFD.
- LabelColumn: Column name of label in your CSV file. This is a mandatory column for AFD. 
    **note: the profiler will identify the "rare" case and assume that it is fraud**
- FileDelimiter: S3 path to your CSV file. E.g. s3://mybucket/myfile.csv
- ProfileCSV: Do you want to profile your data? This will generate an HTML report of your data statistics and potential AFD validation errors.
- FeatureCorr: Do you want to show pair-wise feature correlation in report? The correlation shows that for each pair of features how much one feature depends on the other. This calculation may take some time.
- ReportSuffix: (Optional) Suffix name of profiling report. The report will be named as report_<ReportSuffix>.html.
- FraudLabels: (Optional) Specify label values to be mapped to FRAUD, separated by comma. E.g. suspicious, fraud. If you don't want to map labels, leave this option blank and reprot will show distribution of original label values.

    
</div>

In [ ]:
config = {
    "CSVFilePath": "s3://<filename>",
    "FileDelimiter": ",",
    "EventTimestampColumn": "EVENT_TIMESTAMP",
    "LabelColumn": "EVENT_LABEL",
    "ProfileCSV": "Yes",
    "FraudLabels": " ",
    "FeatureCorr": "Yes",
    "ReportSuffix": ""
}   

#### Run Profiler
-----
The profiler will read your file and produce an HTML file as a result which will be displayed inline within this notebook.  
  
Note: you can also open **report.html** in a separate browser tab. 

In [ ]:
# -- generate the report object --
report = afd_profile.generate_report(config)